In [1]:
# !pip install https://data.pyg.org/whl/torch-2.3.0+cu118/torch_scatter-2.1.2+pt23cu118-cp310-cp310-linux_x86_64.whl
# !pip install https://data.pyg.org/whl/torch-2.3.0+cu118/torch_sparse-0.6.18+pt23cu118-cp310-cp310-linux_x86_64.whl
# !pip install https://data.pyg.org/whl/torch-2.3.0+cu118/torch_cluster-1.6.3+pt23cu118-cp310-cp310-linux_x86_64.whl
# !pip install https://data.pyg.org/whl/torch-2.3.0+cu118/torch_spline_conv-1.2.2+pt23cu118-cp310-cp310-linux_x86_64.whl
# !pip install https://data.pyg.org/whl/torch-2.3.0+cu118/pyg_lib-0.4.0+pt23cu118-cp310-cp310-linux_x86_64.whl

In [9]:
import torch
print(torch.version.cuda)

import torch_geometric
import torch_sparse


10.2


ModuleNotFoundError: No module named 'torch_sparse'

In [7]:
pip install torch-sparse torch-scatter torch-cluster torch-spline-conv

Defaulting to user installation because normal site-packages is not writeable
  Using cached torch_sparse-0.6.12.tar.gz (43 kB)
  Preparing metadata (setup.py) ... done
  ERROR: Command errored out with exit status 1:
   command: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-0dsvombt/torch-sparse_f5cbf8ccba8a4f9fabdeba5d432a802d/setup.py'"'"'; __file__='"'"'/tmp/pip-install-0dsvombt/torch-sparse_f5cbf8ccba8a4f9fabdeba5d432a802d/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /tmp/pip-wheel-ak24dh2n
       cwd: /tmp/pip-install-0dsvombt/torch-sparse_f5cbf8ccba8a4f9fabdeba5d432a802d/
  Complete output (72 lines):
  /home/girishkk/.local/lib/python3.6/site-packages/setuptools/dist.py:726: UserWarni

        self.distribution.run_command(command)
      File "/usr/lib/python3.6/distutils/dist.py", line 974, in run_command
        cmd_obj.run()
      File "/home/girishkk/.local/lib/python3.6/site-packages/setuptools/command/build_ext.py", line 79, in run
        _build_ext.run(self)
      File "/usr/lib/python3.6/distutils/command/build_ext.py", line 339, in run
        self.build_extensions()
      File "/home/girishkk/.local/lib/python3.6/site-packages/torch/utils/cpp_extension.py", line 404, in build_extensions
        self._check_cuda_version()
      File "/home/girishkk/.local/lib/python3.6/site-packages/torch/utils/cpp_extension.py", line 781, in _check_cuda_version
        raise RuntimeError(CUDA_MISMATCH_MESSAGE.format(cuda_str_version, torch.version.cuda))
    RuntimeError:
    The detected CUDA version (11.8) mismatches the version that was used to compile
    PyTorch (10.2). Please make sure to use the same CUDA versions.
    
    ----------------------------------------
E

In [2]:
import torch
import torch_geometric
import torch_scatter
import torch_sparse
import torch_cluster
import torch_spline_conv

/home/girishkk/.local/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'torch_sparse'

In [ ]:
import pandas as pd
import torch
from torch_geometric.data import Data
import networkx as nx
import matplotlib.pyplot as plt
from torch_geometric.utils import to_networkx
from torch_geometric.nn import GATConv
import numpy as np

In [ ]:
file_path = '/creditcard/fraudTrain.csv'  # Update with your .csv file path
df = pd.read_csv(file_path)

In [ ]:
df['Amount'] = df['Amount'].astype(int)

In [ ]:
edge_index = torch.tensor([df['Class'].values, df['Amount'].values], dtype=torch.long)

In [ ]:
num_nodes = int(max(df['Class'].max(), df['Amount'].max()) + 1)
x = torch.eye(num_nodes, dtype=torch.float)  # Example: Identity matrix as node features
y = torch.tensor(df['Class'].values, dtype=torch.long)  # Class labels (0 for non-fraud, 1 for fraud)

In [ ]:
data = Data(x=x, edge_index=edge_index, y=y)

In [ ]:
class Generator(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Generator, self).__init__()
        self.fc1 = torch.nn.Linear(input_size, hidden_size)
        self.fc2 = torch.nn.Linear(hidden_size, output_size)

    def forward(self, z):
        z = torch.relu(self.fc1(z))
        return torch.sigmoid(self.fc2(z))

In [ ]:
class Discriminator(torch.nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Discriminator, self).__init__()
        self.fc1 = torch.nn.Linear(input_size, hidden_size)
        self.fc2 = torch.nn.Linear(hidden_size, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        return torch.sigmoid(self.fc2(x))

In [ ]:
input_size = x.size(1)  # Number of node features
hidden_size = 128
output_size = x.size(1)
latent_size = 64

In [ ]:
generator = Generator(latent_size, hidden_size, output_size)
discriminator = Discriminator(input_size, hidden_size)

In [ ]:
optimizer_g = torch.optim.Adam(generator.parameters(), lr=0.001)
optimizer_d = torch.optim.Adam(discriminator.parameters(), lr=0.001)
loss_fn = torch.nn.BCELoss()

In [ ]:
for epoch in range(10):
    # Discriminator training
    real_data = x[y == 1]  # Fraud samples
    z = torch.randn(real_data.size(0), latent_size)
    fake_data = generator(z)
    d_real = discriminator(real_data)
    d_fake = discriminator(fake_data.detach())
    loss_d = loss_fn(d_real, torch.ones_like(d_real)) + loss_fn(d_fake, torch.zeros_like(d_fake))
    optimizer_d.zero_grad()
    loss_d.backward()
    optimizer_d.step()

    # Generator training
    d_fake = discriminator(fake_data)
    loss_g = loss_fn(d_fake, torch.ones_like(d_fake))
    optimizer_g.zero_grad()
    loss_g.backward()
    optimizer_g.step()

In [ ]:
generated_data = generator(torch.randn(500, latent_size))  # Generate 500 new samples
x_augmented = torch.cat([x, generated_data], dim=0)
y_augmented = torch.cat([y, torch.ones(500, dtype=torch.long)], dim=0)  # Augment with label '1'

In [ ]:
class GAT(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GAT, self).__init__()
        self.conv1 = GATConv(in_channels, 8, heads=8, dropout=0.6)
        self.conv2 = GATConv(8 * 8, out_channels, heads=1, concat=False, dropout=0.6)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = torch.dropout(x, p=0.6, train=self.training)
        x = self.conv2(x, edge_index)
        return torch.log_softmax(x, dim=1)

In [ ]:
model = GAT(input_size, 2)  # Binary classification (fraud or non-fraud)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [ ]:
fraud_count = y_augmented.sum().item()
non_fraud_count = len(y_augmented) - fraud_count

plt.bar(['Non-Fraud', 'Fraud'], [non_fraud_count, fraud_count], color=['blue', 'orange'])
plt.title('Class Distribution After Data Augmentation')
plt.xlabel('Class')
plt.ylabel('Count')
plt.show()

# Step 7: Calculate Mean, Variance, and Standard Deviation
mean_val = torch.mean(x_augmented.float(), dim=0)
variance_val = torch.var(x_augmented.float(), dim=0)
std_val = torch.std(x_augmented.float(), dim=0)

print(f"Mean: {mean_val}")
print(f"Variance: {variance_val}")
print(f"Standard Deviation: {std_val}")